In [1]:
import pandas as pd
import pyodbc

# Specify the connection details
server = '172.16.3.22'
database = 'patentview_trial'
username = 'Raktim_Srivastava'
password = 'IDSL2Sa!gt%1'

# Create a connection string
connection_string = f"Driver={{SQL Server}};Server={server};Database={database};UID={username};PWD={password};"

# Establish the database connection
connection = pyodbc.connect(connection_string)

# Create a cursor object
cursor = connection.cursor()

# Execute the first SQL query
query1 = """
SELECT
    A.patent_id, A.inventor_sequence, A.inventor_id, A.disambig_inventor_name_first, A.disambig_inventor_name_last,
    A.male_flag, A.location_id, B.disambig_city, B.disambig_state, B.disambig_country, B.latitude, B.longitude,
    C.disambig_assignee_individual_name_first, C.disambig_assignee_individual_name_last, C.disambig_assignee_organization,
    D.patent_type, D.patent_title, D.patent_abstract, D.num_claims,
    CASE 
        WHEN SUBSTRING(F.cpc_class, 1, 1) = 'A' THEN 'Human Necessitates'
        WHEN SUBSTRING(F.cpc_class, 1, 1) = 'B' THEN 'Performing Operations Transporting'
        WHEN SUBSTRING(F.cpc_class, 1, 1) = 'C' THEN 'Chemistry Metallurgy'
        WHEN SUBSTRING(F.cpc_class, 1, 1) = 'D' THEN 'Textiles Paper'
        WHEN SUBSTRING(F.cpc_class, 1, 1) = 'E' THEN 'Fixed Constructions'
        WHEN SUBSTRING(F.cpc_class, 1, 1) = 'F' THEN 'Mechanical Engineering Lighting Heating Weapons Blasting Engines or Pumps'
        WHEN SUBSTRING(F.cpc_class, 1, 1) = 'G' THEN 'Physics'
        WHEN SUBSTRING(F.cpc_class, 1, 1) = 'H' THEN 'Electricity'
        WHEN SUBSTRING(F.cpc_class, 1, 1) = 'Y' THEN 'General Tagging of New Technological Developments'
        ELSE F.cpc_class
    END AS PATENT_CATEGORY,
    F.cpc_subclass, F.cpc_subclass_title, F.cpc_group, F.cpc_group_title, F.cpc_class, F.cpc_class_title,
    G.application_id, G.filing_date AS application_filing_date,
    H.priority_claim_kind, H.foreign_application_id, H.filing_date, H.foreign_country_filed
FROM dbo.CAPS_POC_USPTO_inventor_disambiguated A
INNER JOIN [dbo].[USPTO_g_location_disambiguated_TT] B ON A.location_id = B.location_id
LEFT JOIN [dbo].[USPTO_g_assignee_disambiguated_TT] C ON C.patent_id = A.patent_id
LEFT JOIN [dbo].[USPTO_g_patent_TT] D ON D.patent_id = A.patent_id
LEFT JOIN [dbo].USPTO_g_cpc_current_TT E ON E.patent_id = A.patent_id AND E.cpc_sequence = '0'
LEFT JOIN [dbo].USPTO_g_cpc_title_TT F ON F.cpc_class = E.cpc_class AND F.cpc_subclass = E.cpc_subclass AND F.cpc_group = E.cpc_group
LEFT JOIN [dbo].USPTO_g_application_TT G ON G.patent_id = A.patent_id
LEFT JOIN [dbo].USPTO_g_foreign_priority_MAX_REC_TT H ON H.patent_id = A.patent_id
"""

cursor.execute(query1)

# Fetch the first query results
results1 = cursor.fetchall()

# Get the column names from the cursor description
columns1 = [column[0] for column in cursor.description]

# Convert the first query results to a DataFrame
df1 = pd.DataFrame.from_records(results1, columns=columns1)

# Execute the second SQL query
query2 = """
SELECT
    A.entity_id AS author_id, A.class AS Author_class, A.org_memberof, A.foaf_name AS Author_Name, A.paperCount,
    A.paperFamilyCount, A.citationCount, B.paper AS Paper_Id, B.affiliation1, C.foaf_name AS Affliation_Name,
    C.foundation_date, C.type_entities, C.pos_lat, C.pos_long, C.city_name, C.city_lat, C.city_lon, C.state_name,
    C.postcode, C.country_name, C.country_alpha2, C.country_alpha3, C.country_official_name, D.class AS Paper_Class,
    D.entity_type Paper_Entity_Type, D.appearsInJournal, D.estimatedCitationCount, D.referenceCount,
    D.dcterms_created, D.dcterms_title, D.dcterms_publisher, D.dbo_publisher, F.foaf_name AS Field_of_Study_Name,
    G.fos_list AS Field_list
FROM [dbo].[CAPS_POC_EMAKG_Authors] A
INNER JOIN [dbo].[EMAKG_PaperAuthorAffiliation_TT] B ON A.entity_id = B.author
LEFT JOIN [dbo].[EMAKG_Affiliations_TT] C ON C.entity_id = B.affiliation1
INNER JOIN [dbo].[EMAKG_Papers_TT] D ON D.entity_id = B.paper
LEFT JOIN [dbo].[EMAKG_PaperFieldsOfStudy_TT] E ON E.entity_id = B.paper
LEFT JOIN [dbo].[EMAKG_FieldsOfStudy_TT] F ON F.entity_id = E.fabio_hasDiscipline
LEFT JOIN [dbo].[EMAKG_FieldOfStudyLabeled_TT] G ON G.entity_id = F.entity_id
"""

cursor.execute(query2)

# Fetch the second query results
results2 = cursor.fetchall()

# Get the column names from the cursor description
columns2 = [column[0] for column in cursor.description]

# Convert the second query results to a DataFrame
df2 = pd.DataFrame.from_records(results2, columns=columns2)

# Close the cursor and connection
cursor.close()
connection.close()


## Building Matching Code

In [2]:
df1.head()

,patent_id,inventor_sequence,inventor_id,disambig_inventor_name_first,disambig_inventor_name_last,male_flag,location_id,disambig_city,disambig_state,disambig_country,...,cpc_group,cpc_group_title,cpc_class,cpc_class_title,application_id,application_filing_date,priority_claim_kind,foreign_application_id,filing_date,foreign_country_filed
0,10000000,0,fl:jo_ln:marron-5,Joseph,Marron,1.0,198b0471-16c8-11ed-9b5f-1234bde3cd05,Manhattan Beach,CA,US,...,G01S7/4863,Details of systems according to groups G01S13/...,G01,MEASURING; TESTING,2015/14643719,2015-03-10,None,None,None,None
1,10000001,1,fl:hy_ln:yu-30,Hyeon Jae,YU,NaN,3eb37495-16c8-11ed-9b5f-1234bde3cd05,Ansan-si,None,KR,...,B29C45/64,"Injection moulding, i.e. forcing the required ...",B29,WORKING OF PLASTICS; WORKING OF SUBSTANCES IN ...,2015/14962323,2015-12-08,national,10-2014-0187779,2014-12-24,KR
2,10000001,0,fl:su_ln:lee-389,Sun-Woo,Lee,NaN,6c4ba08f-16c8-11ed-9b5f-1234bde3cd05,Gunpo-si,None,KR,...,B29C45/64,"Injection moulding, i.e. forcing the required ...",B29,WORKING OF PLASTICS; WORKING OF SUBSTANCES IN ...,2015/14962323,2015-12-08,national,10-2014-0187779,2014-12-24,KR
3,10000002,2,fl:do_ln:choi-31,Dong-Hyeon,Choi,1.0,755a6338-16c8-11ed-9b5f-1234bde3cd05,Yongin-si,None,KR,...,B29C48/21,"Extrusion moulding, i.e. expressing the mouldi...",B29,WORKING OF PLASTICS; WORKING OF SUBSTANCES IN ...,2014/15107519,2014-12-30,national,10-2013-0169223,2013-12-31,KR
4,10000002,3,fl:do_ln:kim-369,Dong Jin,Kim,1.0,755a6338-16c8-11ed-9b5f-1234bde3cd05,Yongin-si,None,KR,...,B29C48/21,"Extrusion moulding, i.e. expressing the mouldi...",B29,WORKING OF PLASTICS; WORKING OF SUBSTANCES IN ...,2014/15107519,2014-12-30,national,10-2013-0169223,2013-12-31,KR


In [3]:
# Concatenate the first and last names
df1["full_name"] = df1["disambig_inventor_name_first"] + " " + df1["disambig_inventor_name_last"]

# Convert non-string values to strings in the 'full_name' column
df1['full_name'] = df1['full_name'].astype(str)

In [4]:
import re
# Remove punctuation and spaces from the 'full_name' column
df1['clean_name'] = df1['full_name'].apply(lambda x: re.sub(r'[^\w\s]', '', x).replace(" ", ""))

In [5]:
df1.head()

,patent_id,inventor_sequence,inventor_id,disambig_inventor_name_first,disambig_inventor_name_last,male_flag,location_id,disambig_city,disambig_state,disambig_country,...,cpc_class,cpc_class_title,application_id,application_filing_date,priority_claim_kind,foreign_application_id,filing_date,foreign_country_filed,full_name,clean_name
0,10000000,0,fl:jo_ln:marron-5,Joseph,Marron,1.0,198b0471-16c8-11ed-9b5f-1234bde3cd05,Manhattan Beach,CA,US,...,G01,MEASURING; TESTING,2015/14643719,2015-03-10,None,None,None,None,Joseph Marron,JosephMarron
1,10000001,1,fl:hy_ln:yu-30,Hyeon Jae,YU,NaN,3eb37495-16c8-11ed-9b5f-1234bde3cd05,Ansan-si,None,KR,...,B29,WORKING OF PLASTICS; WORKING OF SUBSTANCES IN ...,2015/14962323,2015-12-08,national,10-2014-0187779,2014-12-24,KR,Hyeon Jae YU,HyeonJaeYU
2,10000001,0,fl:su_ln:lee-389,Sun-Woo,Lee,NaN,6c4ba08f-16c8-11ed-9b5f-1234bde3cd05,Gunpo-si,None,KR,...,B29,WORKING OF PLASTICS; WORKING OF SUBSTANCES IN ...,2015/14962323,2015-12-08,national,10-2014-0187779,2014-12-24,KR,Sun-Woo Lee,SunWooLee
3,10000002,2,fl:do_ln:choi-31,Dong-Hyeon,Choi,1.0,755a6338-16c8-11ed-9b5f-1234bde3cd05,Yongin-si,None,KR,...,B29,WORKING OF PLASTICS; WORKING OF SUBSTANCES IN ...,2014/15107519,2014-12-30,national,10-2013-0169223,2013-12-31,KR,Dong-Hyeon Choi,DongHyeonChoi
4,10000002,3,fl:do_ln:kim-369,Dong Jin,Kim,1.0,755a6338-16c8-11ed-9b5f-1234bde3cd05,Yongin-si,None,KR,...,B29,WORKING OF PLASTICS; WORKING OF SUBSTANCES IN ...,2014/15107519,2014-12-30,national,10-2013-0169223,2013-12-31,KR,Dong Jin Kim,DongJinKim


In [6]:
df_usp_final = df1[["full_name","disambig_country","disambig_city","disambig_assignee_organization"]]
df_usp_final.head()


,full_name,disambig_country,disambig_city,disambig_assignee_organization
0,Joseph Marron,US,Manhattan Beach,Raytheon Company
1,Hyeon Jae YU,KR,Ansan-si,LS MTRON LTD.
2,Sun-Woo Lee,KR,Gunpo-si,LS MTRON LTD.
3,Dong-Hyeon Choi,KR,Yongin-si,KOLON INDUSTRIES. INC.
4,Dong Jin Kim,KR,Yongin-si,KOLON INDUSTRIES. INC.


In [7]:
# Rename the columns
df_usp_final = df_usp_final.rename(columns={
    "full_name": "name",
    "disambig_country": "country",
    "disambig_city": "city",
    "Gender" : "gender",
    "disambig_assignee_organization" : "affiliation"

})

In [8]:
#making final df 
df_usp_final.head()

,name,country,city,affiliation
0,Joseph Marron,US,Manhattan Beach,Raytheon Company
1,Hyeon Jae YU,KR,Ansan-si,LS MTRON LTD.
2,Sun-Woo Lee,KR,Gunpo-si,LS MTRON LTD.
3,Dong-Hyeon Choi,KR,Yongin-si,KOLON INDUSTRIES. INC.
4,Dong Jin Kim,KR,Yongin-si,KOLON INDUSTRIES. INC.


In [9]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [11]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 12.8/12.8 MB 7.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
import spacy

nlp = spacy.load('en_core_web_sm')

def perform_ner(text):
    if text is None:
        return ""  # Replace None with an empty string
    doc = nlp(text)
    entities = [ ent.label_ for ent in doc.ents]
    return entities


# Example usage
column_name = 'affiliation'  # Replace with the column name containing the text


# Apply NER to the specified column
df_usp_final['NER_AFF'] = df_usp_final[column_name].fillna("").apply(perform_ner)




# Display the DataFrame with NER results
print(df_usp_final)


                             name country             city  \
0                   Joseph Marron      US  Manhattan Beach   
1                    Hyeon Jae YU      KR         Ansan-si   
2                     Sun-Woo Lee      KR         Gunpo-si   
3                 Dong-Hyeon Choi      KR        Yongin-si   
4                    Dong Jin Kim      KR        Yongin-si   
5                      Si Min Kim      KR        Yongin-si   
6                      Yun-Jo Kim      KR        Yongin-si   
7                Carsten Elsasser      DE          Pulheim   
8                 Cristoph Mehren      DE        Rübhausen   
9                  Guido Bergmann      DE   Sankt Augustin   
10        Jose Juan Valadez Lopez      MX        Monterrey   
11  Miguel Jorge Zubiria Elizondo      MX        Monterrey   
12                  Katsunori Oda      JP           Kariya   
13                    Kenji Katou      JP           Kariya   
14                    Marc Saelen      FR           Carvin   
15      

In [ ]:
df_usp_final.head()

,name,country,city,affiliation,NER_AFF
0,Joseph Marron,US,Manhattan Beach,Raytheon Company,[ORG]
1,Hyeon Jae YU,KR,Ansan-si,LS MTRON LTD.,[ORG]
2,Sun-Woo Lee,KR,Gunpo-si,LS MTRON LTD.,[ORG]
3,Dong-Hyeon Choi,KR,Yongin-si,KOLON INDUSTRIES. INC.,[ORG]
4,Dong Jin Kim,KR,Yongin-si,KOLON INDUSTRIES. INC.,[ORG]


In [ ]:
ner_aff_counts = df_usp_final['NER_AFF'].explode().value_counts()
print(ner_aff_counts)

ORG       21
PERSON    11
Name: NER_AFF, dtype: int64


# Creating City Clusters using Glove 840B Common Crawl Embeddings

## Using Glove embeddings and Kmeans to create clusters

In [17]:
import pickle
import numpy as np
from sklearn.cluster import KMeans

# Step 1: Load pre-trained GloVe embeddings from pickle file
def load_glove_embeddings(file_path):
    with open(file_path, 'rb') as file:
        embeddings = pickle.load(file)
    return embeddings

# Step 2: Load the GloVe embeddings from the pickle file
glove_embeddings = load_glove_embeddings('C:/Users/drgun/OneDrive/Desktop/Capstone/work/glove.840B.300d.pkl')


# Step 3: Preprocess city names
def preprocess_city_name(city_name):
    # Perform any necessary preprocessing (e.g., lowercase, remove punctuation, handle special characters)
    return city_name.lower()

# Step 4: Compute city embeddings
def get_city_embedding(city_name):
    city_name = preprocess_city_name(city_name)
    if city_name in glove_embeddings:
        return glove_embeddings[city_name]
    else:
        return None

# Step 5: Create city codes
def create_city_codes(city_names, num_clusters):
    city_embeddings = []
    valid_city_names = []
    for city_name in city_names:
        embedding = get_city_embedding(city_name)
        if embedding is not None:
            city_embeddings.append(embedding)
            valid_city_names.append(city_name)

    # Perform clustering (K-means) if there are valid city names
    if len(city_embeddings) > 0:
        kmeans = KMeans(n_clusters=num_clusters)
        kmeans.fit(city_embeddings)
        labels = kmeans.labels_

        city_codes = []
        for city_name, label in zip(valid_city_names, labels):
            city_codes.append((city_name, label))

        return city_codes
    else:
        return []


# Step 6: Assign codes to city names
def assign_codes(city_names, city_codes):
    city_code_mapping = {}
    for city_name, code in city_codes:
        city_code_mapping[city_name] = code
    return city_code_mapping

In [18]:


# Example usage
city_names = ['New York', 'London', 'Paris', 'Tokyo', 'Berlin','Kanpur','Malaysia']
city_codes = create_city_codes(city_names, num_clusters=3)
city_code_mapping = assign_codes(city_names, city_codes)

# Print the city codes
for city_name, city_code in city_code_mapping.items():
    print(city_name, city_code)

c:\Users\drgun\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\drgun\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


London 0
Paris 0
Tokyo 0
Berlin 0
Kanpur 1
Malaysia 2


## Using Glove and DBSCAN to create clusters

In [19]:
import pickle
import numpy as np
from sklearn.cluster import KMeans

import string
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')


# Step 1: Load pre-trained GloVe embeddings from pickle file
def load_glove_embeddings(file_path):
    with open(file_path, 'rb') as file:
        embeddings = pickle.load(file)
    return embeddings

# Step 2: Load the GloVe embeddings from the pickle file
glove_embeddings = load_glove_embeddings('C:/Users/drgun/OneDrive/Desktop/Capstone/work/glove.840B.300d.pkl')


# Step 3: Preprocess city names
def preprocess_city_name(city_name):
    # Remove spaces and hyphens
    city_name = city_name.replace(' ', '').replace('-', '')

    # Remove punctuation
    city_name = city_name.translate(str.maketrans('', '', string.punctuation))

    # Remove characters matching regex pattern
    regex_pattern = r'[^\w\s]'
    city_name = re.sub(regex_pattern, '', city_name)

    # Convert to lowercase
    city_name = city_name.lower()

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    city_name = lemmatizer.lemmatize(city_name)

    return city_name

# Step 4: Compute city embeddings
def get_city_embedding(city_name):
    city_name = preprocess_city_name(city_name)
    if city_name in glove_embeddings:
        return glove_embeddings[city_name]
    else:
        return None

# Step 5: Create city codes
def create_city_codes(city_names, num_clusters):
    city_embeddings = []
    valid_city_names = []
    for city_name in city_names:
        embedding = get_city_embedding(city_name)
        if embedding is not None:
            city_embeddings.append(embedding)
            valid_city_names.append(city_name)

    # Perform clustering (K-means) if there are valid city names
    if len(city_embeddings) > 0:
        kmeans = KMeans(n_clusters=num_clusters)
        kmeans.fit(city_embeddings)
        labels = kmeans.labels_

        city_codes = []
        for city_name, label in zip(valid_city_names, labels):
            city_codes.append((city_name, label))

        return city_codes
    else:
        return []


# Step 6: Assign codes to city names
def assign_codes(city_names, city_codes):
    city_code_mapping = {}
    for city_name, code in city_codes:
        city_code_mapping[city_name] = code
    return city_code_mapping

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\drgun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\drgun\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [20]:


# Example usage
city_names = ['New York', 'London', 'Paris', 'Tokyo', 'Berlin','Kanpur','Malaysia','lucknow', 'US','Dubai','USA', 'United-States',"United States"]
city_codes = create_city_codes(city_names, num_clusters=3)
city_code_mapping = assign_codes(city_names, city_codes)

# Print the city codes
for city_name, city_code in city_code_mapping.items():
    print(city_name, city_code)

c:\Users\drgun\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\drgun\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


New York 0
London 0
Paris 0
Tokyo 0
Berlin 0
Kanpur 1
Malaysia 0
lucknow 1
US 0
Dubai 0
USA 0
United-States 2
United States 2


In [23]:
import pickle
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN


import string
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')

# Step 1: Load pre-trained GloVe embeddings from pickle file
def load_glove_embeddings(file_path):
    with open(file_path, 'rb') as file:
        embeddings = pickle.load(file)
    return embeddings

# Step 2: Load the GloVe embeddings from the pickle file
glove_embeddings = load_glove_embeddings('C:/Users/drgun/OneDrive/Desktop/Capstone/work/glove.840B.300d.pkl')

# Step 3: Preprocess city names
# Step 3: Preprocess city names
def preprocess_city_name(city_name):
    # Remove spaces and hyphens
    city_name = city_name.replace(' ', '').replace('-', '')

    # Remove punctuation
    city_name = city_name.translate(str.maketrans('', '', string.punctuation))

    # Remove characters matching regex pattern
    regex_pattern = r'[^\w\s]'
    city_name = re.sub(regex_pattern, '', city_name)

    # Convert to lowercase
    city_name = city_name.lower()

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    city_name = lemmatizer.lemmatize(city_name)

    return city_name

# Step 4: Compute city embeddings
def get_city_embedding(city_name):
    city_name = preprocess_city_name(city_name)
    if city_name in glove_embeddings:
        return glove_embeddings[city_name]
    else:
        return None

# Step 5: Create city codes using DBSCAN
def create_city_codes(city_names, epsilon, min_samples):
    city_embeddings = []
    valid_city_names = []
    for city_name in city_names:
        embedding = get_city_embedding(city_name)
        if embedding is not None:
            city_embeddings.append(embedding)
            valid_city_names.append(city_name)

    # Perform clustering (DBSCAN) if there are valid city names
    if len(city_embeddings) > 0:
        dbscan = DBSCAN(eps=epsilon, min_samples=min_samples, metric='cosine')
        dbscan.fit(city_embeddings)
        labels = dbscan.labels_

        city_codes = []
        for city_name, label in zip(valid_city_names, labels):
            city_codes.append((city_name, label))

        return city_codes
    else:
        return []

# Step 6: Assign codes to city names
def assign_codes(city_names, city_codes):
    city_code_mapping = {}
    for city_name, code in city_codes:
        city_code_mapping[city_name] = code
    return city_code_mapping

# Example usage
city_names = ['New York', 'London', 'Paris', 'Tokyo', 'Berlin', 'Kanpur', 'Malaysia', 'Lucknow', 'US', 'Dubai', 'USA', 'United-States', 'United States']
city_codes = create_city_codes(city_names, epsilon=0.5, min_samples=2)
city_code_mapping = assign_codes(city_names, city_codes)

# Print the city codes
for city_name, city_code in city_code_mapping.items():
    print(city_name, city_code)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\drgun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\drgun\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


New York -1
London 0
Paris 0
Tokyo 0
Berlin 0
Kanpur 1
Malaysia 0
Lucknow 1
US -1
Dubai 0
USA 0
United-States 2
United States 2


In [22]:
# Example usage
city_names = ['New York', 'London', 'Paris', 'Tokyo', 'Berlin', 'Kanpur', 'Malaysia', 'Lucknow', 'US', 'Dubai', 'USA', 'United-States', 'United States']
city_codes = create_city_codes(city_names, epsilon=0.5, min_samples=2)
city_code_mapping = assign_codes(city_names, city_codes)

# Print the city codes
for city_name, city_code in city_code_mapping.items():
    print(city_name, city_code)

New York -1
London -1
Paris -1
Tokyo -1
Berlin -1
Kanpur -1
Malaysia -1
Lucknow -1
US -1
Dubai -1
USA -1
United-States 0
United States 0


## Using GLOVE and Agglomerative CLustering to create city clusters

### Below is the final code to approach as agglomerative performs the best when clustering and can be used for both City/Country for both datasets. The code also incorporates any missing city name or country names as well as embeddings where city names are not maatching

In [24]:
import pickle
import numpy as np
from sklearn.cluster import KMeans

import string
import nltk
from nltk.stem import WordNetLemmatizer

import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_distances

nltk.download('wordnet')
nltk.download('omw-1.4')

# Step 1: Load pre-trained GloVe embeddings from pickle file
def load_glove_embeddings(file_path):
    with open(file_path, 'rb') as file:
        embeddings = pickle.load(file)
    return embeddings

# Step 2: Load the GloVe embeddings from the pickle file
glove_embeddings = load_glove_embeddings('C:/Users/drgun/OneDrive/Desktop/Capstone/work/glove.840B.300d.pkl')

# Step 3: Preprocess city names
def preprocess_city_name(city_name):
    if city_name is None:
        return None

    # Remove spaces and hyphens
    city_name = city_name.replace(' ', '').replace('-', '')
 

    # Remove punctuation
    city_name = city_name.translate(str.maketrans('', '', string.punctuation))

    # Remove characters matching regex pattern
    regex_pattern = r'[^\w\s]'
    city_name = re.sub(regex_pattern, '', city_name)

    # Convert to lowercase
    city_name = city_name.lower()

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    city_name = lemmatizer.lemmatize(city_name)

    return city_name

# Step 4: Compute city embeddings
def get_city_embedding(city_name):
    city_name = preprocess_city_name(city_name)
    if city_name in glove_embeddings:
        return glove_embeddings[city_name]
    else:
        return None

# Step 5: Create city codes using hierarchical clustering
def create_city_codes(city_names, num_clusters):
    city_embeddings = []
    valid_city_names = []
    for city_name in city_names:
        embedding = get_city_embedding(city_name)
        if embedding is not None:
            city_embeddings.append(embedding)
            valid_city_names.append(city_name)

    # Perform clustering (hierarchical clustering) if there are valid city names
    if len(city_embeddings) > 0:
        hierarchical_clustering = AgglomerativeClustering(n_clusters=num_clusters)
        labels = hierarchical_clustering.fit_predict(city_embeddings)

        city_codes = []
        for city_name, label in zip(valid_city_names, labels):
            city_codes.append((city_name, label))

        return city_codes
    else:
        return []

# Step 6: Assign codes to city names
def assign_codes(city_names, city_codes):
    city_code_mapping = {}
    for city_name, code in city_codes:
        city_code_mapping[city_name] = code
    return city_code_mapping

# Example usage
city_names = ['New York', 'London', 'Paris', 'Tokyo', 'Berlin', 'Kanpur', 'Malaysia', 'Lucknow', 'US', 'Dubai', 'USA', 'United-States', 'United States']
city_codes = create_city_codes(city_names, num_clusters=5)
city_code_mapping = assign_codes(city_names, city_codes)

# Print the city codes
for city_name, city_code in city_code_mapping.items():
    print(city_name, city_code)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\drgun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\drgun\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


New York 0
London 0
Paris 0
Tokyo 0
Berlin 0
Kanpur 4
Malaysia 1
Lucknow 4
US 3
Dubai 1
USA 1
United-States 2
United States 2


In [25]:
# Example usage
city_names = ['New York', 'London', 'Paris', 'Tokyo', 'Berlin', 'Kanpur', 'Malaysia', 'Lucknow', 'US', 'Dubai', 'USA', 'United-States', 'United States']
city_codes = create_city_codes(city_names, num_clusters=5)
city_code_mapping = assign_codes(city_names, city_codes)

# Print the city codes
for city_name, city_code in city_code_mapping.items():
    print(city_name, city_code)

New York 0
London 0
Paris 0
Tokyo 0
Berlin 0
Kanpur 4
Malaysia 1
Lucknow 4
US 3
Dubai 1
USA 1
United-States 2
United States 2


### Below code is to use the clustering to assign codes to final dataframe

In [26]:
import pandas as pd

# Assuming your DataFrame is called 'df' and the city column is 'City'

# Extract the city names from the 'City' column
city_names = df_usp_final['city'].tolist()

# Apply the city clustering code to create city codes
city_codes = create_city_codes(city_names, num_clusters=5)

# Create a dictionary mapping city names to codes
city_code_mapping = assign_codes(city_names, city_codes)

# Create a new column 'CityCode' in the DataFrame
df_usp_final['CityCode'] = df_usp_final['city'].map(city_code_mapping)

# Print the DataFrame with the new 'CityCode' column
df_usp_final.head()

,name,country,city,affiliation,NER_AFF,CityCode
0,Joseph Marron,US,Manhattan Beach,Raytheon Company,[ORG],NaN
1,Hyeon Jae YU,KR,Ansan-si,LS MTRON LTD.,[ORG],NaN
2,Sun-Woo Lee,KR,Gunpo-si,LS MTRON LTD.,[ORG],NaN
3,Dong-Hyeon Choi,KR,Yongin-si,KOLON INDUSTRIES. INC.,[ORG],NaN
4,Dong Jin Kim,KR,Yongin-si,KOLON INDUSTRIES. INC.,[ORG],NaN


### Below code helps check how many embeddings are matching in a particular column with Glove

In [29]:
# Assuming your DataFrame is called 'df' and the city column is named 'City'
city_names_df = df_usp_final['city'].tolist()

# Set to store unique missing cities and present cities
missing_cities = set()
present_cities = set()

# Iterate over the city names in the DataFrame and check if they are in the GloVe embeddings
for city_name in city_names_df:
    preprocessed_city_name = preprocess_city_name(city_name)
    if preprocessed_city_name in glove_embeddings:
        present_cities.add(city_name)
    else:
        missing_cities.add(city_name)

# Print the missing cities
print("Missing Cities:")
for city_name in missing_cities:
    print(city_name)

# Print the present cities
print("Present Cities:")
for city_name in present_cities:
    print(city_name)

# Print summary
print("Summary:")
print("Total cities:", len(missing_cities)+len(present_cities))
print("Missing cities:", len(missing_cities))
print("Present cities:", len(present_cities))

Missing Cities:
Pulheim
Ansan-si
Rübhausen
Sankt Augustin
Nümbrecht
Bad Kötzting
West Allis
Gunpo-si
Bad Dürkheim
Manhattan Beach
Lichtenfels
Yongin-si
Present Cities:
Grafton
Webster
Monterrey
Closter
Ontario
Cedarburg
Neustadt
Walworth
Kariya
Milwaukee
Rochester
Cambridge
Pepperell
Carvin
Hayward
Brookfield
Summary:
Total cities: 28
Missing cities: 12
Present cities: 16


In [30]:
# Assuming your DataFrame is called 'df' and the city column is named 'City'
city_names_df = df_usp_final['country'].tolist()

# Set to store unique missing cities and present cities
missing_cities = set()
present_cities = set()

# Iterate over the city names in the DataFrame and check if they are in the GloVe embeddings
for city_name in city_names_df:
    preprocessed_city_name = preprocess_city_name(city_name)
    if preprocessed_city_name in glove_embeddings:
        present_cities.add(city_name)
    else:
        missing_cities.add(city_name)

# Print the missing cities
print("Missing Cities:")
for city_name in missing_cities:
    print(city_name)

# Print the present cities
print("Present Cities:")
for city_name in present_cities:
    print(city_name)

# Print summary
print("Summary:")
print("Total cities:", len(missing_cities)+len(present_cities))
print("Missing cities:", len(missing_cities))
print("Present cities:", len(present_cities))


Missing Cities:
Present Cities:
US
JP
FR
MX
KR
DE
Summary:
Total cities: 6
Missing cities: 0
Present cities: 6


### From the above sample USPTO dataset As you can see 16 out of 12 city names are matching from Glove and all 6 country are present in Glove

## Second DF_EMAKG

## Lets repeat the same process for MAKG for Glove embeddings matching

In [31]:
# Print the second DataFrame
print("Second Query Results:")
df2.head()

Second Query Results:


,author_id,Author_class,org_memberof,Author_Name,paperCount,paperFamilyCount,citationCount,Paper_Id,affiliation1,Affliation_Name,...,Paper_Entity_Type,appearsInJournal,estimatedCitationCount,referenceCount,dcterms_created,dcterms_title,dcterms_publisher,dbo_publisher,Field_of_Study_Name,Field_list
0,2784860342,Author,241749,Joyce Jaeyun Kim,2,2,1,2785120038,241749,University of Cambridge,...,None,NaN,0,0,2018-02-02,Rooted Cosmopolitanism: A Theoretical Tool for...,None,None,None,None
1,2533942970,Author,241749,Dong-jin Kim,2,2,6,1883392277,241749,University of Cambridge,...,None,NaN,1,4,2016-06-24,Economic Reform in North Korea: A Dynamic Gene...,"University Library of Munich, Germany",None,None,None
2,2533942970,Author,241749,Dong-jin Kim,2,2,6,2027719038,241749,University of Cambridge,...,JournalArticle,74096415.0,5,3,2016-06-24,Potential economic reforms in North Korea: a d...,Taylor & Francis Journals,None,None,None
3,2274384076,Author,12834331,Dongjin Kim,2,2,17,2145237307,12834331,University of Wyoming,...,JournalArticle,24251679.0,5,21,2016-06-24,Low-storage Runge--Kutta methods for stochasti...,Elsevier Science Publishers B. V.,None,None,None
4,2274384076,Author,12834331,Dongjin Kim,2,2,17,2088605415,12834331,University of Wyoming,...,JournalArticle,148709879.0,12,22,2016-06-24,Numerical study of interacting particles appro...,"Academic Press Professional, Inc.",None,None,None


In [32]:


import re
# Remove punctuation and spaces from the 'full_name' column
df2['clean_name'] = df2['Author_Name'].apply(lambda x: re.sub(r'[^\w\s]', '', x).replace(" ", ""))


In [33]:
#Subsetting required columns
df_makg_final = df2[["Author_Name","country_alpha2","city_name",'Affliation_Name']]
df_makg_final.head()

,Author_Name,country_alpha2,city_name,Affliation_Name
0,Joyce Jaeyun Kim,GB,East of England,University of Cambridge
1,Dong-jin Kim,GB,East of England,University of Cambridge
2,Dong-jin Kim,GB,East of England,University of Cambridge
3,Dongjin Kim,US,Albany County,University of Wyoming
4,Dongjin Kim,US,Albany County,University of Wyoming


In [34]:
# Rename the columns
df_makg_final = df_makg_final.rename(columns={
    "Author_Name": "name",
    "country_alpha2": "country",
    "city_name": "city",
    "Gender" : "gender",
    "Affliation_Name":"affiliation"

})

In [35]:
df_makg_final.head()

,name,country,city,affiliation
0,Joyce Jaeyun Kim,GB,East of England,University of Cambridge
1,Dong-jin Kim,GB,East of England,University of Cambridge
2,Dong-jin Kim,GB,East of England,University of Cambridge
3,Dongjin Kim,US,Albany County,University of Wyoming
4,Dongjin Kim,US,Albany County,University of Wyoming


In [36]:
def preprocess_city_name(city_name):
    if city_name is None:
        return None

    # Remove spaces and hyphens
    #city_name = city_name.replace(' ', '').replace('-', '')
 

    # Remove punctuation
    #city_name = city_name.translate(str.maketrans('', '', string.punctuation))

    # Remove characters matching regex pattern
    #regex_pattern = r'[^\w\s]'
    #city_name = re.sub(regex_pattern, '', city_name)

    # Convert to lowercase
    #city_name = city_name.lower()

    # Lemmatization
    #lemmatizer = WordNetLemmatizer()
    #city_name = lemmatizer.lemmatize(city_name)

    return city_name


In [37]:
# Assuming your DataFrame is called 'df' and the city column is named 'City'
city_names_df = df_makg_final['city'].tolist()

# Set to store unique missing cities and present cities
missing_cities = set()
present_cities = set()

# Iterate over the city names in the DataFrame and check if they are in the GloVe embeddings
for city_name in city_names_df:
    preprocessed_city_name = preprocess_city_name(city_name)
    if preprocessed_city_name in glove_embeddings:
        present_cities.add(city_name)
    else:
        missing_cities.add(city_name)

# Print the missing cities
print("Missing Cities:")
#for city_name in missing_cities:
    #print(city_name)

# Print the present cities
print("Present Cities:")
for city_name in present_cities:
    print(city_name)

# Print summary
print("Summary:")
print("Total cities:", len(missing_cities)+len(present_cities))
print("Missing cities:", len(missing_cities))
print("Present cities:", len(present_cities))

Missing Cities:
Present Cities:
Braunschweig
Chemnitz
Tuebingen
Taipei
Kumamoto
Geneve
Petaling
Ulm
Donnybrook
Darmstadt
Mitte
Alexandria
Sapporo
Stadtlohn
Changwon
Nonsan
Chinju
Changning
Uberlandia
Wroclaw
Washington
Chavannes
Karlsruhe
Qinghe
Chuncheon
Goettingen
Mountainview
Freiburg
Innsbruck
Villigen
Iksan
Gwangju
Calgary
Seoul
Jona
Dresden
Rolle
Glendale
Baltimore
Jonkoping
Siena
Dahlem
Incheon
Matsuyama
Paris
Linz
Cheonan
Asan
Londrina
Ibaraki
Aachen
Bonn
Eisen
Jeddah
Daejeon
Boeblingen
Daegu
Naju
Ulsan
Wuerzburg
Sanaa
Heidelberg
Essonne
Hayama
Kimhae
Buch
Berkeley
Busan
Erlangen
Malibu
Tokyo
Kiel
Passau
Laatzen
Ecublens
Sendai
Halifax
Wonju
Singapore
Dublin
Norfolk
Suncheon
Alsergrund
Hauts-de-Seine
Bern
Schwieberdingen
Machida
Giessen
Laguna
Schaffhausen
Takatsuki
Ingolstadt
Naka
Luebeck
Campinas
Gongju
Pohang
Munich
Brussels
Julich
Richmond
Sandnes
Kunsan
Gumi
Turku
Roscommon
Dilbeek
Yangsan
Anseong
Riga
Summary:
Total cities: 325
Missing cities: 215
Present cities: 110


In [38]:
# Assuming your DataFrame is called 'df' and the city column is named 'City'
city_names_df = df_makg_final['country'].tolist()

# Set to store unique missing cities and present cities
missing_cities = set()
present_cities = set()

# Iterate over the city names in the DataFrame and check if they are in the GloVe embeddings
for city_name in city_names_df:
    preprocessed_city_name = preprocess_city_name(city_name)
    if preprocessed_city_name in glove_embeddings:
        present_cities.add(city_name)
    else:
        missing_cities.add(city_name)

# Print the missing cities
print("Missing Cities:")
for city_name in missing_cities:
    print(city_name)

# Print the present cities
print("Present Cities:")
for city_name in present_cities:
    print(city_name)

# Print summary
print("Summary:")
print("Total cities:", len(missing_cities)+len(present_cities))
print("Missing cities:", len(missing_cities))
print("Present cities:", len(present_cities))

Missing Cities:
None
Present Cities:
MY
BR
CZ
GB
SG
IE
FR
HK
JP
TZ
SE
CH
BE
ES
IL
SA
US
AT
LV
NO
ZA
PH
IN
AU
CA
KE
NZ
CO
IT
CY
PL
KR
YE
TW
CN
FI
VN
BT
DK
DE
KM
AE
Summary:
Total cities: 43
Missing cities: 1
Present cities: 42


#### As you can see for MAKG there are 110 cities present in embeddings while remaining 215 are missing  

### Below COde to assign clusters to both data frames with similar codes

In [39]:
# Create a unique city names DataFrame from both DataFrames
unique_city_names = pd.concat([df_usp_final['city'], df_makg_final['city']]).drop_duplicates().reset_index(drop=True)

# Generate city codes based on the unique city names
city_codes = create_city_codes(unique_city_names, num_clusters=6)

# Create a mapping of city names to codes
city_code_mapping = dict(city_codes)

# Apply city code mapping to both DataFrames
df_usp_final['CityCode'] = df_usp_final['city'].map(city_code_mapping)
df_makg_final['CityCode'] = df_makg_final['city'].map(city_code_mapping)

# Print the DataFrames with city codes


### Below code to assign country codes basis clustering

In [40]:
# Create a unique city names DataFrame from both DataFrames
unique_city_names = pd.concat([df_usp_final['country'], df_makg_final['country']]).drop_duplicates().reset_index(drop=True)

# Generate city codes based on the unique city names
city_codes = create_city_codes(unique_city_names, num_clusters=6)

# Create a mapping of city names to codes
city_code_mapping = dict(city_codes)

# Apply city code mapping to both DataFrames
df_usp_final['CountryCode'] = df_usp_final['country'].map(city_code_mapping)
df_makg_final['CountryCode'] = df_makg_final['country'].map(city_code_mapping)

In [41]:
#Checking if its assigng some codes
print("CityCodeSummary")
print(df_usp_final['CityCode'].count())
print(df_makg_final['CityCode'].count())

print("CountryCodeSummary")
print(df_usp_final['CountryCode'].count())
print(df_makg_final['CountryCode'].count())



df_makg_final.head()

CityCodeSummary
21
9420
CountryCodeSummary
34
22964


,name,country,city,affiliation,CityCode,CountryCode
0,Joyce Jaeyun Kim,GB,East of England,University of Cambridge,NaN,0.0
1,Dong-jin Kim,GB,East of England,University of Cambridge,NaN,0.0
2,Dong-jin Kim,GB,East of England,University of Cambridge,NaN,0.0
3,Dongjin Kim,US,Albany County,University of Wyoming,NaN,0.0
4,Dongjin Kim,US,Albany County,University of Wyoming,NaN,0.0


### Below code to assign NER codes using SPACY for affiliation 

In [42]:
import spacy

nlp = spacy.load('en_core_web_sm')

def perform_ner(text):
    if text is None:
        return ""
    doc = nlp(text)
    entities = [ent.label_ for ent in doc.ents]
    if entities:
        return entities[0]  # Return the first entity label
    else:
        return ""


# Example usage
column_name = 'affiliation'  # Replace with the column name containing the text


# Apply NER to the specified column
df_usp_final['NER_AFF'] = df_usp_final[column_name].fillna("").apply(perform_ner)
df_makg_final['NER_AFF'] = df_usp_final[column_name].fillna("").apply(perform_ner)






In [43]:
ner_aff_counts_usp = df_usp_final['NER_AFF'].explode().value_counts()

print(ner_aff_counts_usp)


ORG       20
           8
PERSON     6
Name: NER_AFF, dtype: int64


In [44]:
df_usp_final.head()

,name,country,city,affiliation,NER_AFF,CityCode,CountryCode
0,Joseph Marron,US,Manhattan Beach,Raytheon Company,ORG,NaN,0
1,Hyeon Jae YU,KR,Ansan-si,LS MTRON LTD.,ORG,NaN,1
2,Sun-Woo Lee,KR,Gunpo-si,LS MTRON LTD.,ORG,NaN,1
3,Dong-Hyeon Choi,KR,Yongin-si,KOLON INDUSTRIES. INC.,ORG,NaN,1
4,Dong Jin Kim,KR,Yongin-si,KOLON INDUSTRIES. INC.,ORG,NaN,1


In [45]:
df_makg_final.head()

,name,country,city,affiliation,CityCode,CountryCode,NER_AFF
0,Joyce Jaeyun Kim,GB,East of England,University of Cambridge,NaN,0.0,ORG
1,Dong-jin Kim,GB,East of England,University of Cambridge,NaN,0.0,ORG
2,Dong-jin Kim,GB,East of England,University of Cambridge,NaN,0.0,ORG
3,Dongjin Kim,US,Albany County,University of Wyoming,NaN,0.0,ORG
4,Dongjin Kim,US,Albany County,University of Wyoming,NaN,0.0,ORG


### Matching

In [ ]:
pip install recordlinkage

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import recordlinkage
import pandas as pd

df_usp_final = df_usp_final.fillna('')
df_makg_final = df_makg_final.fillna('')


columns_to_convert = ['name', 'gender', 'country', 'city', 'affiliation']
df_usp_final[columns_to_convert] = df_usp_final[columns_to_convert].astype(str)
df_makg_final[columns_to_convert] = df_makg_final[columns_to_convert].astype(str)


# Step 2: Define indexing
indexer = recordlinkage.Index()
indexer.block('name')
indexer.sortedneighbourhood('country')
pairs = indexer.index(df_usp_final,df_makg_final)

# Step 3: Define comparison
compare = recordlinkage.Compare()
compare.string('name', 'name', method='jarowinkler', threshold=0.8)
compare.string('gender', 'gender', method='jarowinkler', threshold=0.8)
compare.string('country', 'country', method='jarowinkler', threshold=0.8)
compare.string('city', 'city', method='jarowinkler', threshold=0.8)
compare.string('affiliation', 'affiliation', method='jarowinkler', threshold=0.8)
similarity = compare.compute(pairs, df_usp_final,df_makg_final)

# Step 4: Perform classification
matches = recordlinkage.KMeansClassifier().fit_predict(similarity)

# Step 5: Retrieve matched record pairs
matched_pairs = pairs[matches == 1]

# Step 6: Create DataFrame or write results to a file
matched_records = pd.DataFrame(matched_pairs, columns=['ID_1', 'ID_2'])
matched_records.to_csv('matched_records.csv', index=False)


In [ ]:
import recordlinkage
import pandas as pd

# Fill missing values with empty strings
df_usp_final = df_usp_final.fillna('')
df_makg_final = df_makg_final.fillna('')

# Convert columns to string type
columns_to_convert = ['name', 'gender', 'country', 'city', 'affiliation']
df_usp_final[columns_to_convert] = df_usp_final[columns_to_convert].astype(str)
df_makg_final[columns_to_convert] = df_makg_final[columns_to_convert].astype(str)

# Define indexing
indexer = recordlinkage.Index()
indexer.block('name')
indexer.sortedneighbourhood('country')
pairs = indexer.index(df_usp_final, df_makg_final)

# Define comparison
compare = recordlinkage.Compare()
compare.string('name', 'name', method='jarowinkler', threshold=0.1)
compare.string('gender', 'gender', method='jarowinkler', threshold=0.1)
compare.string('country', 'country', method='jarowinkler', threshold=0.1)
compare.string('city', 'city', method='jarowinkler', threshold=0.1)
compare.string('affiliation', 'affiliation', method='jarowinkler', threshold=0.1)
similarity = compare.compute(pairs, df_usp_final, df_makg_final)

# Perform classification
matches = recordlinkage.KMeansClassifier().fit_predict(similarity)

# Retrieve matched record pairs
matched_pairs = pairs[matches.values]

# Create DataFrame with matched pairs
matched_records = pd.DataFrame(matched_pairs, columns=['ID_1', 'ID_2'])

# Save matched records to a CSV file
matched_records.to_csv('matched_records.csv', index=False)


### Below is the final code to use for dedup

In [ ]:
import recordlinkage
import pandas as pd

# Fill missing values with empty strings
df_usp_final = df_usp_final.fillna('')
df_makg_final = df_makg_final.fillna('')

# Convert columns to string type
columns_to_convert = ['name', 'gender', 'country', 'city', 'affiliation']
df_usp_final[columns_to_convert] = df_usp_final[columns_to_convert].astype(str)
df_makg_final[columns_to_convert] = df_makg_final[columns_to_convert].astype(str)

# Define indexing
indexer = recordlinkage.Index()
indexer.block('name')
indexer.sortedneighbourhood('country')
pairs = indexer.index(df_usp_final, df_makg_final)

# Define comparison
compare = recordlinkage.Compare()
compare.string('name', 'name', method='jarowinkler', threshold=0.1)  # Adjust the threshold as needed
compare.string('gender', 'gender', method='jarowinkler', threshold=0.1)
compare.string('country', 'country', method='jarowinkler', threshold=0.1)
compare.string('city', 'city', method='jarowinkler', threshold=0.1)
compare.string('affiliation', 'affiliation', method='jarowinkler', threshold=0.1)
similarity = compare.compute(pairs, df_usp_final, df_makg_final)

# Perform classification
matches = recordlinkage.KMeansClassifier().fit_predict(similarity)

# Retrieve matched record pairs
matched_indices = matches[matches.get_level_values(0) == 1].get_level_values(1).tolist()

if len(matched_indices) > 0:
    matched_records = []

    for idx in matched_indices:
        id_1 = pairs[idx][0]
        id_2 = pairs[idx][1]
        matched_records.append((id_1, id_2))

    matched_records_df = pd.DataFrame(matched_records, columns=['ID_1', 'ID_2'])
    matched_records_df.to_csv('matched_records.csv', index=False)
else:
    print("No matched pairs found.")


In [ ]:
import pandas as pd

# Read the CSV file
matched_records_df = pd.read_csv('matched_records.csv')

# Fetch the matched records from the original dataframes
for index, row in matched_records_df.iterrows():
    index_1 = row['ID_1']
    index_2 = row['ID_2']
    record_1 = df_usp_final.iloc[index_1]
    record_2 = df_makg_final.iloc[index_2]

    # Print the matched records
    print(f"Matched Pair:")
    print("Record 1:")
    print(record_1)
    print("Record 2:")
    print(record_2)
    print("-------------------")
